In [ ]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from scipy.sparse import csr_matrix
# import faiss

import random
# from sentence_transformers import SentenceTransformer

warnings.filterwarnings(action='ignore')

books = pd.read_csv('/content/drive/MyDrive/books_mod_fin', encoding='utf-8')
books = books.replace({np.nan: 'none'})


books_df_nax=books.dropna(axis='rows')


tfidf = TfidfVectorizer(stop_words='english')

def find_sim_books(U):
    def userbooks(A):
        x=books_df_nax['isbn'].tolist()
        l=len(A)
        BD=[]
        for i in range(l):
            if A[i] in x:
              BD.append(A[i])
        l2=len(BD)
        n=random.randrange(0,l2)
        return BD[n]

    # 사용자의 isbn 데이터를 담은 책 리스트를 input으로 받아 그 리스트 중 하나를 반환하는 함수
    # B = 사용자 데이터에서 랜덤 추출한 책의 isbn 
    B=userbooks(U) 


    def sim_idx_with_A(df,isbn):
        k=df[df['isbn']==isbn]['kdc'].item()
        n=int(df[df['isbn']==isbn]['des_cluster'])
        if n==280:
          m=int(df[df['isbn']==isbn]['tit_cluster'])
          same_clu_books_df=df[df['tit_cluster']==m]
          sen = same_clu_books_df['content']
          kdc_same_vect=[0 for i in range(len(sen))]

          for i in range(len(sen)):
            if same_clu_books_df[i:i+1]['kdc'].item()==k:
              kdc_same_vect[i]=0.2

        else: 
          same_clu_books_df=df[df['des_cluster']==n]
          sen = same_clu_books_df['content']
          kdc_same_vect=[0 for i in range(len(sen))]

          for i in range(len(sen)):
            if same_clu_books_df[i:i+1]['kdc'].item()==k:
              kdc_same_vect[i]=0.05
        


        # 1번 후보 : sentence transformer 유사도 구하기
        # model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        # sen_array = sen.to_numpy()
        # sen_embeddings = model.encode(sen_array)
        same_clu_books_idx=same_clu_books_df.loc[:,'Unnamed: 0'].to_list()
        tit_embeddings=np.load('/content/drive/MyDrive/tit_embeddings.npy')
        des_embeddings=np.load('/content/drive/MyDrive/sen_embeddings.npy')

        if n==280:
          sen_embeddings=tit_embeddings[same_clu_books_idx,:]
        else:
          sen_embeddings=des_embeddings[same_clu_books_idx,:]

        clu_des_sim=cosine_similarity(sen_embeddings,sen_embeddings)

        # kdc 가 같으면 보너스 점수 주기 
    
        # 2번 후보 : 단순 tfidf 활용한 유사도 구하기
        # clu_tfidf = tfidf.fit_transform(sen)
        # clu_des_sim=cosine_similarity(clu_tfidf,clu_tfidf)


        idx=np.where(same_clu_books_df['isbn']==B)[0][0]
        books_sim_vect=clu_des_sim[idx:idx+1]
        sim_vect_plus_w=books_sim_vect+kdc_same_vect
        books_des_sim_idx = sim_vect_plus_w.argsort()[::-1]


        return books_des_sim_idx

    # 사용자 책 리스트에서 랜덤 추출한 책 기준 유사한 책 인덱스 리스트 반환
    sim_books_idx=sim_idx_with_A(books_df_nax,B)

    top_n=20
    top_sim_idx=sim_books_idx[0][:top_n]
    top_sim_idx=top_sim_idx.reshape(-1,)

    sim_books=books_df_nax.iloc[top_sim_idx]
    # 데이터프레임이 content, thumbnail column을 포함한다면
    outputs=sim_books[['isbn','title','author','content','thumbnail','publisher','kdc','price']]
    # outputs=sim_books[['title','author','img_url','publisher','kdc_class_no','price']]
    outputs_dic=outputs.to_dict('records')

    return outputs_dic

In [ ]:
find_sim_books([9788990008343])

[{'isbn': 9788997348756,
  'title': '카페 인테리어 싸게 하기',
  'author': '이민 (지은이)',
  'content': '『카페 인테리어 싸게 하기』는 홍대의 ‘에티오피아’, 효창동 마다가스카르’, 금호동 ‘커피 브레이크 위드 새미’, 당산동 ‘커피 체리’, 삼성동 ‘카페 네프’ 등 인테리어 공사를 직접한 저자의 카페 인테리어 싸게 직접 하는 방법과 현장 사진, 노하우를 소개하고 있다.',
  'thumbnail': 'https://image.aladin.co.kr/product/10630/42/cover/e899734824_2.jpg',
  'publisher': '푸른영토',
  'kdc': '619.5',
  'price': '8900.0'},
 {'isbn': 9788991307520,
  'title': '김현석의 전공일반사회',
  'author': '김현석 (지은이)',
  'content': "중등임용고시 대비 전공일반사회 수험서. 이 책은 7차 교육과정에 맞춘 정치, 사회문화, 법, 경제의 방대한 양을 한 권에 압축 정리하여 수험생들이 효과적으로 학습할 수 있도록 하였다. 특히 중요사항을 표로 보기 좋게 정리하고, '참고'를 통해 놓치기 쉬운 부분을 짚어 주어 학습효과를 더욱 높일 수 있도록...",
  'thumbnail': 'https://image.aladin.co.kr/product/64/78/cover/8991307523_1.jpg',
  'publisher': '북타운',
  'kdc': '330.76',
  'price': '20000.0'},
 {'isbn': 9780578990385,
  'title': 'I love you more than worms love dirt (Hardcover)',
  'author': 'John Sisco (지은이)',
  'content': 'I Love You More Than... creates a happy, loving atmosphere wher